In [6]:
import numpy as np
import cv2
import math
from tkinter import Tk, Canvas, Button, filedialog
from PIL import Image, ImageTk

class HoughTransformApp:
    def __init__(self, root):
        self.root = root
        self.canvas = Canvas(root, width=800, height=600, bg='white')
        self.canvas.pack()

        self.dots = []
        self.edge_image = None
        self.img_tk = None

        Button(root, text="Place Dots", command=self.enable_dot_placement).pack()
        Button(root, text="Detect Lines on Dots", command=self.detect_lines_on_dots).pack()
        Button(root, text="Upload Image", command=self.upload_image).pack()
        Button(root, text="Detect Lines on Image", command=self.detect_lines_on_image).pack()
    
    def enable_dot_placement(self):
        self.canvas.bind("<Button-1>", self.add_dot)

    def add_dot(self, event):
        x, y = event.x, event.y
        self.dots.append((x, y))
        self.canvas.create_oval(x-2, y-2, x+2, y+2, fill='black')

    def detect_lines_on_dots(self):
        if self.dots:
            lines = self.hough_transform(self.dots)
            self.draw_detected_lines(lines)

    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.edge_image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            self.display_image(self.edge_image)

    def detect_lines_on_image(self):
        if self.edge_image is not None:
            edge_points = self.get_edge_points(self.edge_image)
            lines = self.hough_transform(edge_points)
            self.draw_detected_lines(lines)

    def hough_transform(self, points):
        width, height = self.canvas.winfo_width(), self.canvas.winfo_height()
        diag_len = int(math.hypot(width, height))
        rho_range = 2 * diag_len
        theta_range = 180
        accumulator = np.zeros((rho_range, theta_range), dtype=np.int32)
        thetas = np.deg2rad(np.arange(0, theta_range, 1))  # Fine resolution for angles

        # Voting in the accumulator
        for x, y in points:
            for theta_idx, theta in enumerate(thetas):
                rho = int(x * math.cos(theta) + y * math.sin(theta)) + diag_len
                if 0 <= rho < rho_range:
                    accumulator[rho, theta_idx] += 1

        # Detecting lines above threshold with non-maximum suppression
        threshold = accumulator.max() * 0.95  # Significantly higher threshold for strong filtering
        min_distance = 20  # Minimum distance between lines to avoid duplicates
        lines = []
        for rho_idx in range(accumulator.shape[0]):
            for theta_idx in range(accumulator.shape[1]):
                if accumulator[rho_idx, theta_idx] >= threshold:
                    rho = rho_idx - diag_len
                    theta = theta_idx
                    # Check for close duplicate lines
                    if all(abs(rho - r) > min_distance or abs(theta - t) > min_distance for r, t in lines):
                        lines.append((rho, theta))
                    if len(lines) > 5:  # Limit to a maximum of 5 lines
                        return lines
        return lines

    def draw_detected_lines(self, lines):
        for rho, theta in lines:
            self.draw_line(rho, theta)

    def draw_line(self, rho, theta):
        theta_rad = np.deg2rad(theta)
        cos_theta = math.cos(theta_rad)
        sin_theta = math.sin(theta_rad)

        x0 = rho * cos_theta
        y0 = rho * sin_theta
        x1 = int(x0 + 1000 * (-sin_theta))
        y1 = int(y0 + 1000 * (cos_theta))
        x2 = int(x0 - 1000 * (-sin_theta))
        y2 = int(y0 - 1000 * (cos_theta))

        self.canvas.create_line(x1, y1, x2, y2, fill='blue')

    def display_image(self, image):
        img = Image.fromarray(image)
        self.img_tk = ImageTk.PhotoImage(img)
        self.canvas.create_image(0, 0, anchor="nw", image=self.img_tk)

    def get_edge_points(self, image):
        points = []
        edges = cv2.Canny(image, 50, 150)
        for y in range(edges.shape[0]):
            for x in range(edges.shape[1]):
                if edges[y, x] == 255:
                    points.append((x, y))
        return points

if __name__ == "__main__":
    root = Tk()
    app = HoughTransformApp(root)
    root.mainloop()
